# Binomial (CRR) convergence and American early exercise

This demo shows how the Cox–Ross–Rubinstein (CRR) binomial model behaves as the number of time steps increases, and how the same tree supports **American early exercise**.

**You will:**
- Price a European call with **Black–Scholes** and with the **binomial tree**.
- Run a convergence experiment as the number of steps \(N\) increases.
- (Bonus) Compare **European vs American** puts and quantify the early-exercise premium.

> The demo uses the library’s public entrypoints: `bs_price(...)` and `binom_price(...)`.


## Setup

Imports and a small plotting helper. The notebook is robust: if an optional diagnostics plotting module is not present, it falls back to an inline implementation.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

from option_pricing import (
    MarketData,
    OptionSpec,
    OptionType,
    PricingInputs,
    binom_price,
    bs_price,
)

# Optional: use a reusable diagnostics plotter if available; otherwise use a local fallback.
try:
    from option_pricing.diagnostics.binom_convergence_plots import (
        plot_binom_convergence_to_bs,  # type: ignore
    )
except Exception:
    def plot_binom_convergence_to_bs(
        p: PricingInputs,
        *,
        n_steps_max: int = 500,
        step: int = 5,
        tol: float | None = 1e-2,
        method: str = "tree",
        err_scale: str = "log",
        figsize: tuple[float, float] = (12, 4),
    ):
        """Fallback plotter: price and absolute error vs N."""
        n_steps_vals = np.arange(step, n_steps_max + 1, step=step, dtype=int)
        binom_vals = np.array([binom_price(p, int(n), method=method) for n in n_steps_vals], dtype=float)
        bs_val = float(bs_price(p))
        abs_err = np.abs(binom_vals - bs_val)

        fig, (ax_price, ax_err) = plt.subplots(1, 2, figsize=figsize)

        ax_price.plot(n_steps_vals, binom_vals, label=f"Binomial ({method})")
        ax_price.hlines(bs_val, n_steps_vals.min(), n_steps_vals.max(), linestyles="dashed", label="Black–Scholes")
        ax_price.set_xlabel("Number of steps $N$")
        ax_price.set_ylabel("Option price")
        ax_price.set_title("Convergence of binomial price to Black–Scholes")
        ax_price.legend()
        ax_price.grid(True, alpha=0.25)

        # Avoid issues with log-scale when error hits 0 exactly
        abs_err_plot = np.maximum(abs_err, np.finfo(float).tiny) if err_scale == "log" else abs_err

        ax_err.plot(n_steps_vals, abs_err_plot, label="Absolute error")
        if tol is not None and not (err_scale == "log" and tol <= 0):
            ax_err.hlines(tol, n_steps_vals.min(), n_steps_vals.max(), linestyles="dashed", label=f"Tolerance = {tol:.2g}")
        ax_err.set_xlabel("Number of steps $N$")
        ax_err.set_ylabel("Absolute error")
        ax_err.set_yscale(err_scale)
        ax_err.set_title("Error decay with $N$")
        ax_err.legend()
        ax_err.grid(True, which="both", ls=":", alpha=0.35)

        fig.tight_layout()
        return fig, (ax_price, ax_err), {
            "n_steps": n_steps_vals,
            "binom": binom_vals,
            "bs": bs_val,
            "abs_error": abs_err,
        }


## Inputs

We use one set of market inputs and price both a call and a put with the same strike and expiry.

In [ ]:
# Base inputs used throughout the demo
market = MarketData(spot=100.0, rate=0.02, dividend_yield=0.0)

spec_call = OptionSpec(kind=OptionType.CALL, strike=100.0, expiry=1.0)
spec_put  = OptionSpec(kind=OptionType.PUT,  strike=100.0, expiry=1.0)

p_call = PricingInputs(spec=spec_call, market=market, sigma=0.20, t=0.0)
p_put  = PricingInputs(spec=spec_put,  market=market, sigma=0.20, t=0.0)

p_call, p_put


## Quick check: BS vs binomial (European call)

The binomial pricer has two European methods:
- `method='tree'`: backward induction (also supports American)
- `method='closed_form'`: fast binomial sum (European only)

For European options, both methods should agree (up to numerical tolerance) and converge towards Black–Scholes as \(N\) increases.

In [ ]:
N = 200

bs = bs_price(p_call)
tree_price = binom_price(p_call, N, method="tree", american=False)
sum_price  = binom_price(p_call, N, method="closed_form", american=False)

print(f"Black–Scholes (call):        {bs:.6f}")
print(f"Binomial tree (N={N}):       {tree_price:.6f}")
print(f"Binomial closed-form (N={N}): {sum_price:.6f}")
print(f"|tree - BS|: {abs(tree_price - bs):.6g}")
print(f"|tree - sum|: {abs(tree_price - sum_price):.6g}")

# Sanity: European tree and closed-form should be close for the same N
assert abs(tree_price - sum_price) < 5e-3, "Tree and closed-form should match closely for European options."


## Convergence experiment

We compute the binomial price for a range of step counts \(N\) and compare to Black–Scholes. The absolute error typically decays as \(N\) increases.

In [ ]:
fig, (ax_price, ax_err), data = plot_binom_convergence_to_bs(
    p_call,
    n_steps_max=500,
    step=5,
    tol=1e-2,
    method="tree",
    err_scale="log",
)
plt.show()

# Small helper: smallest N achieving a given tolerance (if any)
tol = 1e-2
n_steps = data["n_steps"]
abs_err = np.asarray(data["abs_error"], dtype=float)

idx = np.where(abs_err <= tol)[0]
if idx.size > 0:
    n_star = int(n_steps[idx[0]])
    print(f"First N with abs error ≤ {tol:.2g}: N = {n_star}")
else:
    print(f"No N up to {int(n_steps.max())} achieved abs error ≤ {tol:.2g}.")


### Convergence table

A compact table is often easier to scan than a plot.

In [ ]:
Ns = [5, 10, 25, 50, 100, 250, 500]
rows = []
bs_val = float(bs_price(p_call))

for n in Ns:
    price_n = float(binom_price(p_call, int(n), method="tree", american=False))
    rows.append((n, price_n, bs_val, abs(price_n - bs_val)))

df = pd.DataFrame(rows, columns=["N", "binom_tree", "bs_price", "abs_error"]).set_index("N")

# Pretty formatting for display
display(df.style.format({
    "binom_tree": "{:.6f}",
    "bs_price": "{:.6f}",
    "abs_error": "{:.6g}",
}))


## Bonus: American put early-exercise premium

A key advantage of the tree method is that it supports early exercise. For non-dividend-paying stocks (\(q=0\)), an **American put** is typically worth more than the corresponding European put.

In [ ]:
N = 300

euro_put = float(binom_price(p_put, N, american=False, method="tree"))
amer_put = float(binom_price(p_put, N, american=True, method="tree"))

premium = amer_put - euro_put

print(f"European put (tree, N={N}): {euro_put:.6f}")
print(f"American put  (tree, N={N}): {amer_put:.6f}")
print(f"Premium (Amer - Euro):      {premium:.6g}")

assert amer_put >= euro_put - 1e-12, "American option should be worth at least the European."


### Premium vs number of steps

The estimated premium can vary with \(N\); increasing \(N\) typically stabilizes the result.

In [ ]:
Ns = [25, 50, 100, 200, 300, 500]

rows = []
for n in Ns:
    euro = float(binom_price(p_put, int(n), american=False, method="tree"))
    amer = float(binom_price(p_put, int(n), american=True, method="tree"))
    rows.append((n, euro, amer, amer - euro))

prem_df = pd.DataFrame(rows, columns=["N", "euro_put", "amer_put", "premium"]).set_index("N")

display(prem_df.style.format({
    "euro_put": "{:.6f}",
    "amer_put": "{:.6f}",
    "premium": "{:.6g}",
}))


## Takeaways

- For European options under GBM, **binomial prices converge** to Black–Scholes as the number of steps increases.
- The binomial implementation provides both:
  - a **tree** method (supports American exercise),
  - a **closed-form** binomial sum (European only, faster).
- For puts, **American ≥ European**, and the difference is the **early-exercise premium**.
